In [1]:
import copy
import math

import mrcfile
import numba
import numpy as np
import pyfftw
from numba.typed import List
from tqdm.notebook import tqdm

# from Bio.PDB import *
# import numba as nb

In [3]:
class mrc_obj:
    def __init__(self, path):
        mrc = mrcfile.open(path)
        data = mrc.data
        header = mrc.header
        self.xdim = int(header.nx)
        self.ydim = int(header.ny)
        self.zdim = int(header.nz)
        self.xwidth = mrc.voxel_size.x
        self.ywidth = mrc.voxel_size.y
        self.zwidth = mrc.voxel_size.z
        self.cent = [
            self.xdim * 0.5,
            self.ydim * 0.5,
            self.zdim * 0.5,
        ]
        self.orig = {"x": header.origin.x, "y": header.origin.y, "z": header.origin.z}
        self.data = copy.deepcopy(data)
        self.dens = data.flatten()
        self.vec = np.zeros((self.xdim, self.ydim, self.zdim, 3), dtype="float32")
        self.dsum = None
        self.Nact = None
        self.ave = None
        self.std_norm_ave = None
        self.std = None

In [4]:
def mrc_set_vox_size(mrc, th=0.01, voxel_size=7.0):

    # set shape and size
    size = mrc.xdim * mrc.ydim * mrc.zdim
    shape = (mrc.xdim, mrc.ydim, mrc.zdim)

    # if th < 0 add th to all value
    if th < 0:
        mrc.dens = mrc.dens - th
        th = 0.0

    # Trim all the values less than threshold
    mrc.dens[mrc.dens < th] = 0.0
    mrc.data[mrc.data < th] = 0.0

    # calculate dmax distance for non-zero entries
    non_zero_index_list = np.array(np.nonzero(mrc.data)).T
    non_zero_index_list[:, [2, 0]] = non_zero_index_list[:, [0, 2]]
    cent_arr = np.array(mrc.cent)
    d2_list = np.linalg.norm(non_zero_index_list - cent_arr, axis=1)
    dmax = max(d2_list)

    # dmax = math.sqrt(mrc.cent[0] ** 2 + mrc.cent[1] ** 2 + mrc.cent[2] ** 2)

    print("#dmax=" + str(dmax / mrc.xwidth))
    dmax = dmax * mrc.xwidth

    # set new center
    new_cent = [
        mrc.cent[0] * mrc.xwidth + mrc.orig["x"],
        mrc.cent[1] * mrc.xwidth + mrc.orig["y"],
        mrc.cent[2] * mrc.xwidth + mrc.orig["z"],
    ]

    tmp_size = 2 * dmax / voxel_size

    # find the minimum size of the map
    b = y = 2 ** math.ceil(math.log2(tmp_size))
    while 1:
        while y < tmp_size:
            y = y * 3
            continue
        if y < b:
            b = y
        if y % 2 != 0:
            break
        y = y / 2

    new_xdim = int(b)

    # set new origins
    new_orig = {
        "x": new_cent[0] - 0.5 * new_xdim * voxel_size,
        "y": new_cent[1] - 0.5 * new_xdim * voxel_size,
        "z": new_cent[2] - 0.5 * new_xdim * voxel_size,
    }

    # create new mrc object
    mrc_set = copy.deepcopy(mrc)
    mrc_set.orig = new_orig
    mrc_set.xdim = mrc_set.ydim = mrc_set.zdim = new_xdim
    mrc_set.cent = new_cent
    mrc_set.xwidth = mrc_set.ywidth = mrc_set.zwidth = voxel_size
    mrc_set.dens = np.zeros((mrc_set.xdim ** 3, 1))
    mrc_set.vec = np.zeros((new_xdim, new_xdim, new_xdim, 3), dtype="float32")

    print(
        "Nvox= "
        + str(mrc_set.xdim)
        + ", "
        + str(mrc_set.ydim)
        + ", "
        + str(mrc_set.zdim)
    )
    print(
        "cent= " + str(new_cent[0]) + ", " + str(new_cent[1]) + ", " + str(new_cent[2])
    )
    print(
        "ori= "
        + str(new_orig["x"])
        + ", "
        + str(new_orig["y"])
        + ", "
        + str(new_orig["z"])
    )

    return mrc, mrc_set

In [5]:
@numba.jit(nopython=True)
def calc(stp, endp, pos, mrc1_data, fsiv):
    dtotal = 0
    pos2 = [0.0] * 3

    for xp in range(stp[0], endp[0]):
        rx = float(xp) - pos[0]
        rx = rx ** 2
        for yp in range(stp[1], endp[1]):
            ry = float(yp) - pos[1]
            ry = ry ** 2
            for zp in range(stp[2], endp[2]):
                rz = float(zp) - pos[2]
                rz = rz ** 2
                d2 = rx + ry + rz
                v = mrc1_data[zp][yp][xp] * math.exp(-1.5 * d2 * fsiv)
                dtotal += v
                pos2[0] += v * float(xp)
                pos2[1] += v * float(yp)
                pos2[2] += v * float(zp)
    return dtotal, pos2

In [6]:
def fastVEC(mrc1, mrc2, dreso=16.0):

    xydim = mrc1.xdim * mrc1.ydim
    Ndata = mrc2.xdim * mrc2.ydim * mrc2.zdim

    print(len(mrc2.dens))

    print("#Start VEC")
    gstep = mrc1.xwidth
    fs = (dreso / gstep) * 0.5
    fs = fs ** 2
    fsiv = 1.0 / fs
    fmaxd = (dreso / gstep) * 2.0
    print("#maxd= {fmaxd}".format(fmaxd=fmaxd))
    print("#fsiv= " + str(fsiv))

    dsum = 0.0
    Nact = 0

    list_d = []

    for x in tqdm(range(mrc2.xdim)):
        for y in range(mrc2.ydim):
            for z in range(mrc2.zdim):
                stp = [0] * 3
                endp = [0] * 3
                ind2 = 0
                ind = 0

                pos = [0.0] * 3
                pos2 = [0.0] * 3
                ori = [0.0] * 3

                tmpcd = [0.0] * 3

                v, dtotal, rd = 0.0, 0.0, 0.0

                pos[0] = (
                    x * mrc2.xwidth + mrc2.orig["x"] - mrc1.orig["x"]
                ) / mrc1.xwidth
                pos[1] = (
                    y * mrc2.xwidth + mrc2.orig["y"] - mrc1.orig["y"]
                ) / mrc1.xwidth
                pos[2] = (
                    z * mrc2.xwidth + mrc2.orig["z"] - mrc1.orig["z"]
                ) / mrc1.xwidth

                ind = mrc2.xdim * mrc2.ydim * z + mrc2.xdim * y + x

                # check density

                if (
                    pos[0] < 0
                    or pos[1] < 0
                    or pos[2] < 0
                    or pos[0] >= mrc1.xdim
                    or pos[1] >= mrc1.ydim
                    or pos[2] >= mrc1.zdim
                ):
                    mrc2.dens[ind] = 0.0
                    mrc2.vec[x][y][z][0] = 0.0
                    mrc2.vec[x][y][z][1] = 0.0
                    mrc2.vec[x][y][z][2] = 0.0
                    continue

                if mrc1.data[int(pos[2])][int(pos[1])][int(pos[0])] == 0:
                    mrc2.dens[ind] = 0.0
                    mrc2.vec[x][y][z][0] = 0.0
                    mrc2.vec[x][y][z][1] = 0.0
                    mrc2.vec[x][y][z][2] = 0.0
                    continue

                ori[0] = pos[0]
                ori[1] = pos[1]
                ori[2] = pos[2]

                # Start Point
                stp[0] = int(pos[0] - fmaxd)
                stp[1] = int(pos[1] - fmaxd)
                stp[2] = int(pos[2] - fmaxd)

                # set start and end point
                if stp[0] < 0:
                    stp[0] = 0
                if stp[1] < 0:
                    stp[1] = 0
                if stp[2] < 0:
                    stp[2] = 0

                endp[0] = int(pos[0] + fmaxd + 1)
                endp[1] = int(pos[1] + fmaxd + 1)
                endp[2] = int(pos[2] + fmaxd + 1)

                if endp[0] >= mrc1.xdim:
                    endp[0] = mrc1.xdim
                if endp[1] >= mrc1.ydim:
                    endp[1] = mrc1.ydim
                if endp[2] >= mrc1.zdim:
                    endp[2] = mrc1.zdim

                # setup for numba acc
                stp_t = List()
                endp_t = List()
                pos_t = List()
                [stp_t.append(x) for x in stp]
                [endp_t.append(x) for x in endp]
                [pos_t.append(x) for x in pos]

                # compute the total density
                dtotal, pos2 = calc(stp_t, endp_t, pos_t, mrc1.data, fsiv)

                # list_d.append(dtotal)

                mrc2.dens[ind] = dtotal

                if dtotal == 0:
                    mrc2.vec[x][y][z][0] = 0.0
                    mrc2.vec[x][y][z][1] = 0.0
                    mrc2.vec[x][y][z][2] = 0.0
                    continue

                rd = 1.0 / dtotal

                pos2[0] *= rd
                pos2[1] *= rd
                pos2[2] *= rd

                tmpcd[0] = pos2[0] - pos[0]
                tmpcd[1] = pos2[1] - pos[1]
                tmpcd[2] = pos2[2] - pos[2]

                dvec = math.sqrt(tmpcd[0] ** 2 + tmpcd[1] ** 2 + tmpcd[2] ** 2)

                if dvec == 0:
                    dvec = 1.0

                rdvec = 1.0 / dvec

                mrc2.vec[x][y][z][0] = tmpcd[0] * rdvec
                mrc2.vec[x][y][z][1] = tmpcd[1] * rdvec
                mrc2.vec[x][y][z][2] = tmpcd[2] * rdvec

                dsum += dtotal
                Nact += 1

    print("#End LDP")
    print(dsum)
    print(Nact)

    mrc2.dsum = dsum
    mrc2.Nact = Nact
    mrc2.ave = dsum / float(Nact)
    mrc2.std = np.linalg.norm(mrc2.dens[mrc2.dens > 0])
    mrc2.std_norm_ave = np.linalg.norm(mrc2.dens[mrc2.dens > 0] - mrc2.ave)
    mrc2.data = np.zeros((mrc2.zdim, mrc2.ydim, mrc2.xdim))

    print(
        "#MAP AVE={ave} STD={std} STD_norm={std_norm}".format(
            ave=mrc2.ave, std=mrc2.std, std_norm=mrc2.std_norm_ave
        )
    )
    # return False
    return mrc2

In [ ]:
def search_map_fft(mrc1, mrc2, ang, is_eval_mode, TopN):

    if is_eval_mode:
        print("#For Evaluation Mode")
        print("#Please use the same coordinate system and map size for map1 and map2.")
        print("#Example:")
        print("#In Chimera command line: open map1 and map2 as #0 and #1, then type")
        print("#> open map1.mrc")
        print("#> open map2.mrc")
        print("#> vop #1 resample onGrid #0")
        print("#> volume #2 save new.mrc")
        print("#Chimera will generate the resampled map2.mrc as new.mrc")
        return

In [7]:
mrc1 = mrc_obj("./data/emd_8097.mrc")
mrc2 = mrc_obj("./data/ChainA_simulated_resample.mrc")

mrc1, mrc_N1 = mrc_set_vox_size(mrc1)
mrc2, mrc_N2 = mrc_set_vox_size(mrc2)

if mrc_N1.xdim > mrc_N2.xdim:
    mrc_N2.xdim = mrc_N2.ydim = mrc_N2.zdim = mrc_N1.xdim

    mrc_N2.orig["x"] = mrc_N2.cent[0] - 0.5 * 7 * mrc_N2.xdim
    mrc_N2.orig["y"] = mrc_N2.cent[1] - 0.5 * 7 * mrc_N2.xdim
    mrc_N2.orig["z"] = mrc_N2.cent[2] - 0.5 * 7 * mrc_N2.xdim

else:
    mrc_N1.xdim = mrc_N1.ydim = mrc_N1.zdim = mrc_N2.xdim

    mrc_N1.orig["x"] = mrc_N1.cent[0] - 0.5 * 7 * mrc_N1.xdim
    mrc_N1.orig["y"] = mrc_N1.cent[1] - 0.5 * 7 * mrc_N1.xdim
    mrc_N1.orig["z"] = mrc_N1.cent[2] - 0.5 * 7 * mrc_N1.xdim

#dmax=141.34333822979175
Nvox= 48, 48, 48
cent= 118.23501896858215, 107.53501725196838, 102.1850163936615
ori= -49.76498103141785, -60.464982748031616, -65.8149836063385
#dmax=128.68973561744724
Nvox= 48, 48, 48
cent= 118.23501896858215, 107.53501725196838, 102.1850163936615
ori= -49.76498103141785, -60.464982748031616, -65.8149836063385


In [8]:
mrc_N1 = fastVEC(mrc1, mrc_N1)

110592
#Start VEC
#maxd= 29.9065372581333
#fsiv= 0.017889068239927756


  0%|          | 0/48 [00:00<?, ?it/s]

#End LDP
32939.518123120644
2257
#MAP AVE=14.594381091325053 STD=773.4275314049188 STD_norm=342.72184914544823


In [9]:
mrc_N2 = fastVEC(mrc2, mrc_N2)

110592
#Start VEC
#maxd= 29.9065372581333
#fsiv= 0.017889068239927756


  0%|          | 0/48 [00:00<?, ?it/s]

#End LDP
77063.58651876262
557
#MAP AVE=138.35473342686288 STD=3523.8520239494 STD_norm=1324.9230607740571


In [13]:
mrc_N1.vec[17][17][12][0]

0.35054788